In [1]:
"""
Created on Tue Mar 14 13:20:15 2017

@author: Mattijn van Hoek, HKV Lijn in water
"""

'\nCreated on Tue Mar 14 13:20:15 2017\n\n@author: Mattijn van Hoek, HKV Lijn in water\n'

In [2]:
from osgeo import gdal
import geopandas as gpd
import os
import subprocess as sp
import re

In [211]:
# cmd function, input is a comma separated list of items
def cmd(command):
    print (sp.list2cmdline(command))
    norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True).communicate()    
    return norm

In [233]:
def tif2pts(a,y,z):
    """
    input:
    z is shapefile containing features to clip
    a is file to read as input raster
    y is folder to store the output files
    output:
    -
    """
    
    # read the baggervakken shapefile into a geodataframe
    gdf = gpd.read_file(z)

    # iterate over each feature in the geodataframe and apply cut procedure
    for idx in range(len(gdf))[38:39]:
        print (idx)
        feature = gdf.ix[idx]
        obID   = feature['OBJECTCODE']
        obNAME = feature['OBJECTNAAM']
        obID_obName = obID+' '+obNAME
        obID_obName = re.sub('[^a-zA-Z0-9 \n\.]', '', obID_obName)
        
        # crop original tif of waddenzee to cutline of each feature and store as VRT file
        b = os.path.join(y, obID_obName+'.vrt')
        print(b)
        # 8 Only keep the ArcInfoASCIIs that contains data
        command = gdalwarp+' -overwrite -of VRT -crop_to_cutline -cutline '+z+' -cwhere '+'"'+'OBJECTNAAM='+"'"+obNAME+"'"'" '+a+' "'+b+'"'
        print (command)
        norm = sp.Popen(command, stdout=sp.PIPE, shell=True).communicate()    

        # use the VRT file to convert to XYZ file
        c = os.path.join(y, obID_obName+'.xyz')
        norm = cmd([gdal_translate, '-of', 'XYZ', b, c])

        # remove all NaN values from the XYZ file
        d = os.path.join(y, obID_obName+'.tmp')
        command = grep+' -v " -999" '+'"'+c+'" > "'+d+'"'+' && move '+'"'+d+'" '+'"'+c+'"'
        print (command)
        norm = sp.Popen(command, stdout=sp.PIPE, shell=True).communicate()

        # remove the VRT file and continue to next feature
        os.remove(b)
        # if there is still a tmp file, then both the tmp and xyz file can be removed
        if os.path.isfile(d):
            os.remove(d)
            os.remove(c)
    return norm,(print('function was succesfull'))

In [234]:
# set some fixed paths to executables to use
gdalwarp = r'C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'
grep = r'"C:\Program Files (x86)\GnuWin32\bin//grep.exe"'

z = r'D:\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Baggervakken\Baggervakken.shp'
y = r'D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden'
#a = r'D:\Projects\Pr\3317.20\TIF2PTS\actuele_bodemhoogte.tif'
a = r'D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte.tif'

In [235]:
# apply the function to get XYZ files from TIF files
norm = tif2pts(a,y,z)

38
D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden\1.7.1.3.2.2 Den Oever Buitenhaven.vrt
C:\Python35\Lib\site-packages\osgeo//gdalwarp.exe -overwrite -of VRT -crop_to_cutline -cutline D:\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Baggervakken\Baggervakken.shp -cwhere "OBJECTNAAM='Den Oever, Buitenhaven'" D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte.tif "D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden\1.7.1.3.2.2 Den Oever Buitenhaven.vrt"
C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe -of XYZ "D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden\1.7.1.3.2.2 Den Oever Buitenhaven.vrt" "D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden\1.7.1.3.2.2 Den Oever Buitenhaven.xyz"
"C:\Program Files (x86)\GnuWin32\bin//grep.exe" -v " -999" "D:\OmsWaddenzee\trunk\fews\Modules\netcdfExpor

In [225]:
norm

((b'Input file size is 1299, 1957\r\n0...10...20...30...40...50...60...70...80...90...100 - done.\r\n',
  None),
 None)

In [176]:
C:\Python35\python.exe 
D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\script\tif2pts.py 
--shp_file D:\OmsWaddenzee\trunk\fews\Config\MapLayerFiles\Baggervakken\Baggervakken.shp 
--xyz_folder D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte_xyz_bestanden 
--tif_file D:\OmsWaddenzee\trunk\fews\Modules\netcdfExport\exportOMS\actuele_bodemhoogte.tif

'\nC:\\Python35\\python.exe \nD:\\OmsWaddenzee\trunk\x0cews\\Modules\netcdfExport\\script\tif2pts.py \n--shp_file D:\\OmsWaddenzee\trunk\x0cews\\Config\\MapLayerFiles\\Baggervakken\\Baggervakken.shp \n--xyz_folder D:\\OmsWaddenzee\trunk\x0cews\\Modules\netcdfExport\\exportOMS\x07ctuele_bodemhoogte_xyz_bestanden \n--tif_file D:\\OmsWaddenzee\trunk\x0cews\\Modules\netcdfExport\\exportOMS\x07ctuele_bodemhoogte.tif\n'

In [177]:
a,y,z

('D:\\Projects\\Pr\\3317.20\\TIF2PTS\\actuele_bodemhoogte.tif',
 'D:\\Projects\\Pr\\3317.20\\TIF2PTS\\pts',
 'D:\\OmsWaddenzee\\trunk\\fews\\Config\\MapLayerFiles\\Baggervakken\\Baggervakken.shp')